In [1]:
! pip install geopy

Looking in indexes: http://jfrog.fkinternal.com/artifactory/api/pypi/python_virtual/simple


In [18]:
!pip install scikit-learn-extra


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: http://jfrog.fkinternal.com/artifactory/api/pypi/python_virtual/simple


In [1]:
import torch
from transformers import AutoModel, AutoTokenizer
import numpy as np
import pickle
from sklearn.neighbors import KNeighborsClassifier
from sklearn_extra.cluster import KMedoids
import pandas as pd
from tqdm import tqdm
import h3
from geopy.distance import geodesic
import os
import time

/home/uddeshya.singh/miniconda3/envs/uddeshya_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#checkpoint_path = "/home/devanapalli.ravi/disk_1/paper/data/gpu_re_trained_models/lm_retrained_models/pan_india_train_threshold/pan_india_train_threshold_out_dir_5_epochs/checkpoint-375000"  
# checkpoint_path = "Experiments/punjab_roberta_model_without_city_05/checkpoint_epoch2"

checkpoint_path = '/home/uddeshya.singh/DataSet_triplet/Roberta_classification_r'
def load_finetuned_model(checkpoint_path):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = AutoModel.from_pretrained(checkpoint_path)
    tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
    model = model.to(device)
    print("Loading done")
    return model, tokenizer, device

model, tokenizer, device = load_finetuned_model(checkpoint_path)

Some weights of RobertaModel were not initialized from the model checkpoint at /home/uddeshya.singh/DataSet_triplet/Roberta_classification_r and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loading done


In [29]:
#input_pickle_path = 'DataSet_triplet/PUNJAB_05_without_city.pkl'
#input_pickle_path = 'DataSet_triplet/classification_dataset/PUNJAB_2_for_lookup_without_city.pkl'
input_pickle_path = "DataSet_triplet/PUNJAB_1_for_lookup_without_city.pkl"
with open(input_pickle_path, 'rb') as f:
    data = pickle.load(f)

# triplets_5 = data.get('triplets', [])
addresses_5 = data.get('addresses', [])
lat_lngs_5 = data.get('lat_lngs', [])

# print(f"Length of triplets: {len(triplets_5)}, {triplets_5[-1]}")
print(f"Length of addresses: {len(addresses_5)}, {addresses_5[-1]}")
print(f"Length of lat_lngs: {len(lat_lngs_5)}, {lat_lngs_5[-1]}")

Length of addresses: 7438504, wo sukhdev singh village chak guru po sarhan renuan banga nawanshahr behram sbs nagar
Length of lat_lngs: 7438504, [31.204298600000005, 75.8766702]


In [4]:
#embeddings_save_path = 'DataSet_triplet/embeddings_ANN_2_without_city_Roberta_classification_R.pkl'
embeddings_save_path = "DataSet_triplet/embeddings_ANN_1_without_city_Roberta_classification.pkl"
#embeddings_save_path = 'DataSet_triplet/embeddings_ANN_5_without_city_pan_india_test_base_roberta_4_epoch4.pkl'
with open(embeddings_save_path, 'rb') as f:
    loaded_embeddings = pickle.load(f)

print(f"Loaded embeddings shape: {loaded_embeddings.shape}")

Loaded embeddings shape: (7438504, 768)


In [5]:
print('hi')

hi


In [6]:
def get_batch_embeddings(model, tokenizer, device, texts):
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).cpu().numpy()


def compute_embeddings_in_batches(model, tokenizer, device, addresses, batch_size=128):
    all_embeddings = []
    for i in tqdm(range(0, len(addresses), batch_size), desc="Computing embeddings"):
        batch_texts = addresses[i:i + batch_size]
        batch_embeddings = get_batch_embeddings(model, tokenizer, device, batch_texts)
        all_embeddings.append(batch_embeddings)
    return np.vstack(all_embeddings)

In [7]:
test_csv_path = '/home/uddeshya.singh/DataSet_triplet/combined_cleaned_triplet_test_PUNJAB_full_1.csv'  # Update this path
test_df = pd.read_csv(test_csv_path)

In [8]:
len(test_df)

204872

In [9]:
test_df.head()

city  pincode  \
0           Pathankot   145025   
1  Pathankot District   145025   
2  Pathankot District   145025   
3           Pathankot   145025   
4           Pathankot   145025   

                                     cleaned_address  lookup_lat  lookup_lng  \
0                       0 adda barth sahib Pathankot   32.244419   75.567518   
1   0 barth sahib puli radha swami pawan Pathanko...   32.244132   75.568270   
2   0 old route no nh 15 pathankot subdistrict Pa...   32.244134   75.568292   
3              1 farwaal post office sarna Pathankot   32.259957   75.573422   
4                           1 lahri bawian Pathankot   32.258106   75.545563   

   no_of_delivereies  
0                 21  
1                 36  
2                 30  
3                 20  
4                 29

In [10]:
#test_df2 = test_df[:12800].copy()

In [11]:
test_addresses = test_df['cleaned_address'].tolist()
test_embeddings = compute_embeddings_in_batches(model, tokenizer, device, test_addresses)

Computing embeddings: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1601/1601 [03:13<00:00,  8.26it/s]


In [12]:
len(test_embeddings)

204872

In [73]:
# embeddings_save_path = 'DataSet_triplet/embeddings_model_2_r_test.pkl'

# with open(embeddings_save_path, 'wb') as f:
#     pickle.dump(test_embeddings, f)

# print(f"Embeddings saved to {embeddings_save_path}")

Embeddings saved to DataSet_triplet/embeddings_model_2_r_test.pkl


In [ ]:
# with open(embeddings_save_path, 'rb') as f:
#     test_embeddings = pickle.load(f)

# print(f"Loaded embeddings shape: {test_embeddings.shape}")

In [13]:
test_embeddings.shape

(204872, 768)

In [14]:
print(test_df['cleaned_address'][:5].values)

[' 0 adda barth sahib Pathankot'
 ' 0 barth sahib puli radha swami pawan Pathankot District'
 ' 0 old route no nh 15 pathankot subdistrict Pathankot District'
 ' 1 farwaal post office sarna Pathankot' ' 1 lahri bawian Pathankot']


In [15]:
from sklearn.neighbors import KNeighborsClassifier
import time

In [22]:
k = 3
print('starting now ', time.strftime('%X %x %Z'))
nbrs = KNearestNeighbors(n_neighbors=k).fit(loaded_embeddings)
print('done now ', time.strftime('%X %x %Z'))

starting now  13:30:58 06/29/24 IST


NameError: name 'KNearestNeighbors' is not defined

In [82]:
k = 8
print('starting now ', time.strftime('%X %x %Z'))
nbrs = KNeighborsClassifier(n_neighbors=k)
nbrs.fit(loaded_embeddings, [0] * len(loaded_embeddings)) 
print('done now ', time.strftime('%X %x %Z'))

starting now  12:10:14 07/12/24 IST
done now  12:10:16 07/12/24 IST


In [83]:
test_embeddings.shape

(204872, 768)

In [70]:
distances, indices = nbrs.kneighbors(test_embeddings[0].reshape(1, -1))

In [71]:
distances.shape , indices.shape

((1, 20), (1, 20))

In [84]:
def get_knn_in_batches(knn, test_embeddings, batch_size=512):
    distances_list = []
    indices_list = []
    for i in tqdm(range(0, len(test_embeddings), batch_size), desc="Finding KNN"):
        batch_embeddings = test_embeddings[i:i + batch_size]
        distances, indices = knn.kneighbors(batch_embeddings)
        distances_list.append(distances)
        indices_list.append(indices)
    return np.vstack(distances_list), np.vstack(indices_list)


In [73]:
print('hi')

hi


In [85]:
distances, indices = get_knn_in_batches(nbrs, test_embeddings) ## roberta_classificarion_RV@8 at >=1 

Finding KNN: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 401/401 [1:39:21<00:00, 14.87s/it]


In [74]:
distances, indices = get_knn_in_batches(nbrs, test_embeddings) ## roberta_classificarion_RV@50 at >=1 

Finding KNN: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 401/401 [1:33:14<00:00, 13.95s/it]


In [59]:
distances, indices = get_knn_in_batches(nbrs, test_embeddings) ## roberta_classificarion_RV@50 at >=1 

Finding KNN: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 401/401 [1:33:28<00:00, 13.99s/it]


In [40]:
distances, indices = get_knn_in_batches(nbrs, test_embeddings) ## roberta_classificarion_RV@5 at >=1 @10

Finding KNN: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 801/801 [1:24:49<00:00,  6.35s/it]


In [21]:
distances, indices = get_knn_in_batches(nbrs, test_embeddings) ## roberta_classificarion_RV@5 at >=1

Finding KNN: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 801/801 [1:18:05<00:00,  5.85s/it]


In [22]:
distances, indices = get_knn_in_batches(nbrs, test_embeddings) ## roberta_classificarion_RV@5 at >=2

Finding KNN: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 801/801 [1:08:01<00:00,  5.10s/it]


In [62]:
distances, indices = get_knn_in_batches(nbrs, test_embeddings) ## roberta_classificarion_RV@5 at >=3

Finding KNN: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 801/801 [41:08<00:00,  3.08s/it]


In [32]:
distances, indices = get_knn_in_batches(nbrs, test_embeddings) ## roberta_classificarion_RV@1

Finding KNN: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 801/801 [28:40<00:00,  2.15s/it]


In [18]:
distances, indices = get_knn_in_batches(nbrs, test_embeddings) ## roberta_classificarion_RV@10

Finding KNN: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 801/801 [28:15<00:00,  2.12s/it]


In [103]:
distances, indices = get_knn_in_batches(nbrs, test_embeddings) ## roberta_classificarion_RV@3

Finding KNN: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 801/801 [28:24<00:00,  2.13s/it]


In [84]:
distances, indices = get_knn_in_batches(nbrs, test_embeddings) ## roberta_classificarion_RV@5

Finding KNN: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 801/801 [36:51<00:00,  2.76s/it]


In [52]:
distances, indices = get_knn_in_batches(nbrs, test_embeddings) # embedding r2_50 (1 July 1:45)

Finding KNN: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 801/801 [46:59<00:00,  3.52s/it]


In [16]:
distances, indices = get_knn_in_batches(nbrs, test_embeddings) # embedding r4

Finding KNN:   0%|                                                                                                                                                                                                        | 0/801 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Finding KNN: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 801/801 [46:21<00:00,  3.47s/it]


In [59]:
distances, indices = get_knn_in_batches(nbrs, test_embeddings) # embedding r2.2

Finding KNN: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 801/801 [29:03<00:00,  2.18s/it]


In [19]:
distances, indices = get_knn_in_batches(nbrs, test_embeddings) # embedding r3

Finding KNN: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 801/801 [28:58<00:00,  2.17s/it]


In [67]:
distances, indices = get_knn_in_batches(nbrs, test_embeddings) # embedding r2

Finding KNN: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 801/801 [24:18<00:00,  1.82s/it]


In [77]:
distances, indices = get_knn_in_batches(nbrs, test_embeddings) ## BASE_ROBERTA

Finding KNN: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 801/801 [28:59<00:00,  2.17s/it]


In [86]:
distances.shape , indices.shape

((204872, 8), (204872, 8))

In [87]:
print('hii')

hii


In [88]:
def compute_medoid(lat_lngs):
    kmedoids = KMedoids(n_clusters=1, method='pam', max_iter=0, random_state=0)
    kmedoids.fit(lat_lngs)
    return kmedoids.cluster_centers_[0]

In [89]:
medoids = []
for idx in tqdm(indices):
    neighbor_lat_lngs = [lat_lngs_5[i] for i in idx]
    medoid = compute_medoid(neighbor_lat_lngs)
    medoids.append(medoid)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 204872/204872 [03:25<00:00, 999.02it/s]


In [64]:
neighbor_lat_lngs1 = [lat_lngs_5[i] for i in indices[1]]

In [65]:
neighbor_lat_lngs1

[[32.2447443, 75.5674356],
 [32.245125, 75.5672067],
 [32.245678600000005, 75.5605011],
 [32.2451632, 75.567329],
 [32.245137199999995, 75.5672509],
 [32.2455021, 75.5668151],
 [32.2418572, 75.567847],
 [32.245396899999996, 75.56729498],
 [32.2463902, 75.57189699999998],
 [32.2449571, 75.5674003],
 [32.24551769999999, 75.5675751],
 [32.2002315, 75.4536998],
 [32.244931699999995, 75.56742829999997],
 [32.2450934, 75.5671444],
 [32.2449378, 75.56708090000002],
 [32.2451745, 75.5672775],
 [32.2003156, 75.45361650000002],
 [32.200230499999996, 75.4537005],
 [32.247326300000005, 75.5610323],
 [32.245857199999996, 75.5657773],
 [32.2452161, 75.5672757],
 [32.244595399999994, 75.56751290000003],
 [32.245153, 75.5672438],
 [32.245163, 75.5671728],
 [32.2397723, 75.5677771],
 [32.25788, 75.5465127],
 [32.2512331, 75.54889320000002],
 [32.2239517, 75.5477033],
 [32.247634000000005, 75.55564009999998],
 [32.2471776, 75.57092270000003],
 [32.2629567, 75.58731999999998],
 [32.245226, 75.56732729999

In [37]:
len(medoids)

204872

In [90]:
test_df['pred_lat'] = [medoid[0] for medoid in medoids]
test_df['pred_lng'] = [medoid[1] for medoid in medoids]

In [32]:
test_df['pred_lat'] = [medoid[0] for medoid in medoids]
test_df['pred_lng'] = [medoid[1] for medoid in medoids]

In [112]:
test_df.head() #### roberta_RV@3

city  pincode  \
0           Pathankot   145025   
1  Pathankot District   145025   
2  Pathankot District   145025   
3           Pathankot   145025   
4           Pathankot   145025   

                                     cleaned_address  lookup_lat  lookup_lng  \
0                       0 adda barth sahib Pathankot   32.244419   75.567518   
1   0 barth sahib puli radha swami pawan Pathanko...   32.244132   75.568270   
2   0 old route no nh 15 pathankot subdistrict Pa...   32.244134   75.568292   
3              1 farwaal post office sarna Pathankot   32.259957   75.573422   
4                           1 lahri bawian Pathankot   32.258106   75.545563   

   no_of_delivereies   pred_lat   pred_lng  pred_drift  
0                 21  32.245333  75.566966         100  
1                 36  32.245163  75.567329         144  
2                 30  32.230145  75.550872        3019  
3                 20  32.260501  75.573608          62  
4                 29  32.257893  75.546686         197

In [93]:
test_df.head() #### roberta_RV@5

city  pincode  \
0           Pathankot   145025   
1  Pathankot District   145025   
2  Pathankot District   145025   
3           Pathankot   145025   
4           Pathankot   145025   

                                     cleaned_address  lookup_lat  lookup_lng  \
0                       0 adda barth sahib Pathankot   32.244419   75.567518   
1   0 barth sahib puli radha swami pawan Pathanko...   32.244132   75.568270   
2   0 old route no nh 15 pathankot subdistrict Pa...   32.244134   75.568292   
3              1 farwaal post office sarna Pathankot   32.259957   75.573422   
4                           1 lahri bawian Pathankot   32.258106   75.545563   

   no_of_delivereies   pred_lat   pred_lng  
0                 21  32.245229  75.567045  
1                 36  32.245163  75.567329  
2                 30  32.223681  75.547136  
3                 20  32.260501  75.573608  
4                 29  32.259608  75.546687

In [30]:
test_df.head() #### roberta_RV@1

city  pincode  \
0           Pathankot   145025   
1  Pathankot District   145025   
2  Pathankot District   145025   
3           Pathankot   145025   
4           Pathankot   145025   

                                     cleaned_address  lookup_lat  lookup_lng  \
0                       0 adda barth sahib Pathankot   32.244419   75.567518   
1   0 barth sahib puli radha swami pawan Pathanko...   32.244132   75.568270   
2   0 old route no nh 15 pathankot subdistrict Pa...   32.244134   75.568292   
3              1 farwaal post office sarna Pathankot   32.259957   75.573422   
4                           1 lahri bawian Pathankot   32.258106   75.545563   

   no_of_delivereies   pred_lat   pred_lng  
0                 21  32.247178  75.560603  
1                 36  32.245137  75.567251  
2                 30  32.230145  75.550872  
3                 20  32.261513  75.573480  
4                 29  32.257893  75.546686

In [94]:
test_df['pred_lat'] = [medoid[0] for medoid in medoids]
test_df['pred_lng'] = [medoid[1] for medoid in medoids]

In [33]:
test_df.head()

city  pincode  \
0           Pathankot   145025   
1  Pathankot District   145025   
2  Pathankot District   145025   
3           Pathankot   145025   
4           Pathankot   145025   

                                     cleaned_address  lookup_lat  lookup_lng  \
0                       0 adda barth sahib Pathankot   32.244419   75.567518   
1   0 barth sahib puli radha swami pawan Pathanko...   32.244132   75.568270   
2   0 old route no nh 15 pathankot subdistrict Pa...   32.244134   75.568292   
3              1 farwaal post office sarna Pathankot   32.259957   75.573422   
4                           1 lahri bawian Pathankot   32.258106   75.545563   

   no_of_delivereies   pred_lat   pred_lng  pred_drift  
0                 21  32.247178  75.560603         719  
1                 36  32.245137  75.567251         147  
2                 30  32.230145  75.550872        2258  
3                 20  32.261513  75.573480         172  
4                 29  32.257893  75.546686         108

In [91]:
def get_centroid_of_grid(grid_id_local): ### calssificarion_Roberta_RV_model@8 ref >=1 
    try:
        return h3.h3_to_geo(grid_id_local)
    except Exception as e:
        print("Exception occurred", grid_id_local, e)
        return None

def get_deviation_for_df(lat_local, lng_local, centroid_local):
    try:
        return int(geodesic((lat_local, lng_local), centroid_local).meters)
    except Exception as e:
        print("Exception occurred", e)
        return None

def get_drift_for_row(row_local):
    """get drift between pred coordinates and lookup coordinates"""
    pred_coor = (row_local['pred_lat'], row_local['pred_lng'])
    pred_drift = get_deviation_for_df(row_local["lookup_lat"], row_local["lookup_lng"], pred_coor)
    return pred_drift

def get_accuracy_for_df(drift_list_local):
    accuracy_dict_local = {100: 0, 200: 0, 500: 0, 1000: 0, 2000: 0, "rest": 0, "None": 0}
    for i in drift_list_local:
        if i is None or pd.isnull(i):
            accuracy_dict_local["None"] += 1
            continue

        if i < 100:
            accuracy_dict_local[100] += 1
        elif i < 200:
            accuracy_dict_local[200] += 1
        elif i < 500:
            accuracy_dict_local[500] += 1
        elif i < 1000:
            accuracy_dict_local[1000] += 1
        elif i < 2000:
            accuracy_dict_local[2000] += 1
        else:
            accuracy_dict_local["rest"] += 1

    accuracy_dict_local[200] += accuracy_dict_local[100]
    accuracy_dict_local[500] += accuracy_dict_local[200]
    accuracy_dict_local[1000] += accuracy_dict_local[500]
    accuracy_dict_local[2000] += accuracy_dict_local[1000]

    accuracy_dict_per_local = {100: 0, 200: 0, 500: 0, 1000: 0, 2000: 0, "rest": 0, "None": 0}
    for k, v in accuracy_dict_local.items():
        accuracy_dict_per_local[k] = float(v) / max(1, (accuracy_dict_local[2000] + accuracy_dict_local["rest"]))

    final_accuracy_dict_local = {'num': accuracy_dict_local, 'per': accuracy_dict_per_local}
    return final_accuracy_dict_local

test_df['pred_drift'] = test_df.apply(lambda row: get_drift_for_row(row), axis=1)
test_drift_acc_dict = get_accuracy_for_df(test_df['pred_drift'].tolist())

print(test_drift_acc_dict)

{'num': {100: 153021, 200: 171549, 500: 185898, 1000: 191482, 2000: 194701, 'rest': 10171, 'None': 0}, 'per': {100: 0.7469102659221367, 200: 0.8373472216798782, 500: 0.9073860752079347, 1000: 0.9346421180053888, 2000: 0.9503543676051388, 'rest': 0.04964563239486118, 'None': 0.0}}


In [80]:
def get_centroid_of_grid(grid_id_local): ### calssificarion_Roberta_RV_model@20 ref >=1 
    try:
        return h3.h3_to_geo(grid_id_local)
    except Exception as e:
        print("Exception occurred", grid_id_local, e)
        return None

def get_deviation_for_df(lat_local, lng_local, centroid_local):
    try:
        return int(geodesic((lat_local, lng_local), centroid_local).meters)
    except Exception as e:
        print("Exception occurred", e)
        return None

def get_drift_for_row(row_local):
    """get drift between pred coordinates and lookup coordinates"""
    pred_coor = (row_local['pred_lat'], row_local['pred_lng'])
    pred_drift = get_deviation_for_df(row_local["lookup_lat"], row_local["lookup_lng"], pred_coor)
    return pred_drift

def get_accuracy_for_df(drift_list_local):
    accuracy_dict_local = {100: 0, 200: 0, 500: 0, 1000: 0, 2000: 0, "rest": 0, "None": 0}
    for i in drift_list_local:
        if i is None or pd.isnull(i):
            accuracy_dict_local["None"] += 1
            continue

        if i < 100:
            accuracy_dict_local[100] += 1
        elif i < 200:
            accuracy_dict_local[200] += 1
        elif i < 500:
            accuracy_dict_local[500] += 1
        elif i < 1000:
            accuracy_dict_local[1000] += 1
        elif i < 2000:
            accuracy_dict_local[2000] += 1
        else:
            accuracy_dict_local["rest"] += 1

    accuracy_dict_local[200] += accuracy_dict_local[100]
    accuracy_dict_local[500] += accuracy_dict_local[200]
    accuracy_dict_local[1000] += accuracy_dict_local[500]
    accuracy_dict_local[2000] += accuracy_dict_local[1000]

    accuracy_dict_per_local = {100: 0, 200: 0, 500: 0, 1000: 0, 2000: 0, "rest": 0, "None": 0}
    for k, v in accuracy_dict_local.items():
        accuracy_dict_per_local[k] = float(v) / max(1, (accuracy_dict_local[2000] + accuracy_dict_local["rest"]))

    final_accuracy_dict_local = {'num': accuracy_dict_local, 'per': accuracy_dict_per_local}
    return final_accuracy_dict_local

test_df['pred_drift'] = test_df.apply(lambda row: get_drift_for_row(row), axis=1)
test_drift_acc_dict = get_accuracy_for_df(test_df['pred_drift'].tolist())

print(test_drift_acc_dict)

{'num': {100: 150452, 200: 170586, 500: 185384, 1000: 191042, 2000: 194271, 'rest': 10601, 'None': 0}, 'per': {100: 0.7343707290405717, 200: 0.8326467257604748, 500: 0.9048771916123238, 1000: 0.932494435550002, 2000: 0.9482554961146472, 'rest': 0.0517445038853528, 'None': 0.0}}


In [67]:
def get_centroid_of_grid(grid_id_local): ### calssificarion_Roberta_RV_model@50 ref >=1 
    try:
        return h3.h3_to_geo(grid_id_local)
    except Exception as e:
        print("Exception occurred", grid_id_local, e)
        return None

def get_deviation_for_df(lat_local, lng_local, centroid_local):
    try:
        return int(geodesic((lat_local, lng_local), centroid_local).meters)
    except Exception as e:
        print("Exception occurred", e)
        return None

def get_drift_for_row(row_local):
    """get drift between pred coordinates and lookup coordinates"""
    pred_coor = (row_local['pred_lat'], row_local['pred_lng'])
    pred_drift = get_deviation_for_df(row_local["lookup_lat"], row_local["lookup_lng"], pred_coor)
    return pred_drift

def get_accuracy_for_df(drift_list_local):
    accuracy_dict_local = {100: 0, 200: 0, 500: 0, 1000: 0, 2000: 0, "rest": 0, "None": 0}
    for i in drift_list_local:
        if i is None or pd.isnull(i):
            accuracy_dict_local["None"] += 1
            continue

        if i < 100:
            accuracy_dict_local[100] += 1
        elif i < 200:
            accuracy_dict_local[200] += 1
        elif i < 500:
            accuracy_dict_local[500] += 1
        elif i < 1000:
            accuracy_dict_local[1000] += 1
        elif i < 2000:
            accuracy_dict_local[2000] += 1
        else:
            accuracy_dict_local["rest"] += 1

    accuracy_dict_local[200] += accuracy_dict_local[100]
    accuracy_dict_local[500] += accuracy_dict_local[200]
    accuracy_dict_local[1000] += accuracy_dict_local[500]
    accuracy_dict_local[2000] += accuracy_dict_local[1000]

    accuracy_dict_per_local = {100: 0, 200: 0, 500: 0, 1000: 0, 2000: 0, "rest": 0, "None": 0}
    for k, v in accuracy_dict_local.items():
        accuracy_dict_per_local[k] = float(v) / max(1, (accuracy_dict_local[2000] + accuracy_dict_local["rest"]))

    final_accuracy_dict_local = {'num': accuracy_dict_local, 'per': accuracy_dict_per_local}
    return final_accuracy_dict_local

test_df['pred_drift'] = test_df.apply(lambda row: get_drift_for_row(row), axis=1)
test_drift_acc_dict = get_accuracy_for_df(test_df['pred_drift'].tolist())

print(test_drift_acc_dict)

{'num': {100: 144327, 200: 167290, 500: 183598, 1000: 189744, 2000: 193200, 'rest': 11672, 'None': 0}, 'per': {100: 0.7044740130422898, 200: 0.816558631731032, 500: 0.8961595532820493, 1000: 0.9261587723066109, 2000: 0.9430278417743762, 'rest': 0.056972158225623806, 'None': 0.0}}


In [45]:
def get_centroid_of_grid(grid_id_local): ### calssificarion_Roberta_RV_model@10 ref >=1 
    try:
        return h3.h3_to_geo(grid_id_local)
    except Exception as e:
        print("Exception occurred", grid_id_local, e)
        return None

def get_deviation_for_df(lat_local, lng_local, centroid_local):
    try:
        return int(geodesic((lat_local, lng_local), centroid_local).meters)
    except Exception as e:
        print("Exception occurred", e)
        return None

def get_drift_for_row(row_local):
    """get drift between pred coordinates and lookup coordinates"""
    pred_coor = (row_local['pred_lat'], row_local['pred_lng'])
    pred_drift = get_deviation_for_df(row_local["lookup_lat"], row_local["lookup_lng"], pred_coor)
    return pred_drift

def get_accuracy_for_df(drift_list_local):
    accuracy_dict_local = {100: 0, 200: 0, 500: 0, 1000: 0, 2000: 0, "rest": 0, "None": 0}
    for i in drift_list_local:
        if i is None or pd.isnull(i):
            accuracy_dict_local["None"] += 1
            continue

        if i < 100:
            accuracy_dict_local[100] += 1
        elif i < 200:
            accuracy_dict_local[200] += 1
        elif i < 500:
            accuracy_dict_local[500] += 1
        elif i < 1000:
            accuracy_dict_local[1000] += 1
        elif i < 2000:
            accuracy_dict_local[2000] += 1
        else:
            accuracy_dict_local["rest"] += 1

    accuracy_dict_local[200] += accuracy_dict_local[100]
    accuracy_dict_local[500] += accuracy_dict_local[200]
    accuracy_dict_local[1000] += accuracy_dict_local[500]
    accuracy_dict_local[2000] += accuracy_dict_local[1000]

    accuracy_dict_per_local = {100: 0, 200: 0, 500: 0, 1000: 0, 2000: 0, "rest": 0, "None": 0}
    for k, v in accuracy_dict_local.items():
        accuracy_dict_per_local[k] = float(v) / max(1, (accuracy_dict_local[2000] + accuracy_dict_local["rest"]))

    final_accuracy_dict_local = {'num': accuracy_dict_local, 'per': accuracy_dict_per_local}
    return final_accuracy_dict_local

test_df['pred_drift'] = test_df.apply(lambda row: get_drift_for_row(row), axis=1)
test_drift_acc_dict = get_accuracy_for_df(test_df['pred_drift'].tolist())

print(test_drift_acc_dict)

{'num': {100: 152736, 200: 171502, 500: 185932, 1000: 191484, 2000: 194672, 'rest': 10200, 'None': 0}, 'per': {100: 0.7455191534226249, 200: 0.837117810144871, 500: 0.9075520324885782, 1000: 0.9346518801983678, 2000: 0.9502128158069428, 'rest': 0.04978718419305713, 'None': 0.0}}


In [36]:
def get_centroid_of_grid(grid_id_local): ### calssificarion_Roberta_RV_model@5 ref >=1
    try:
        return h3.h3_to_geo(grid_id_local)
    except Exception as e:
        print("Exception occurred", grid_id_local, e)
        return None

def get_deviation_for_df(lat_local, lng_local, centroid_local):
    try:
        return int(geodesic((lat_local, lng_local), centroid_local).meters)
    except Exception as e:
        print("Exception occurred", e)
        return None

def get_drift_for_row(row_local):
    """get drift between pred coordinates and lookup coordinates"""
    pred_coor = (row_local['pred_lat'], row_local['pred_lng'])
    pred_drift = get_deviation_for_df(row_local["lookup_lat"], row_local["lookup_lng"], pred_coor)
    return pred_drift

def get_accuracy_for_df(drift_list_local):
    accuracy_dict_local = {100: 0, 200: 0, 500: 0, 1000: 0, 2000: 0, "rest": 0, "None": 0}
    for i in drift_list_local:
        if i is None or pd.isnull(i):
            accuracy_dict_local["None"] += 1
            continue

        if i < 100:
            accuracy_dict_local[100] += 1
        elif i < 200:
            accuracy_dict_local[200] += 1
        elif i < 500:
            accuracy_dict_local[500] += 1
        elif i < 1000:
            accuracy_dict_local[1000] += 1
        elif i < 2000:
            accuracy_dict_local[2000] += 1
        else:
            accuracy_dict_local["rest"] += 1

    accuracy_dict_local[200] += accuracy_dict_local[100]
    accuracy_dict_local[500] += accuracy_dict_local[200]
    accuracy_dict_local[1000] += accuracy_dict_local[500]
    accuracy_dict_local[2000] += accuracy_dict_local[1000]

    accuracy_dict_per_local = {100: 0, 200: 0, 500: 0, 1000: 0, 2000: 0, "rest": 0, "None": 0}
    for k, v in accuracy_dict_local.items():
        accuracy_dict_per_local[k] = float(v) / max(1, (accuracy_dict_local[2000] + accuracy_dict_local["rest"]))

    final_accuracy_dict_local = {'num': accuracy_dict_local, 'per': accuracy_dict_per_local}
    return final_accuracy_dict_local

test_df['pred_drift'] = test_df.apply(lambda row: get_drift_for_row(row), axis=1)
test_drift_acc_dict = get_accuracy_for_df(test_df['pred_drift'].tolist())

print(test_drift_acc_dict)

{'num': {100: 152025, 200: 170464, 500: 185126, 1000: 191076, 2000: 194485, 'rest': 10387, 'None': 0}, 'per': {100: 0.7420486938185794, 200: 0.832051231988754, 500: 0.9036178687180288, 1000: 0.9326603928306455, 2000: 0.9493000507634035, 'rest': 0.05069994923659651, 'None': 0.0}}


In [31]:
def get_centroid_of_grid(grid_id_local): ### calssificarion_Roberta_RV_model@5 ref >=2
    try:
        return h3.h3_to_geo(grid_id_local)
    except Exception as e:
        print("Exception occurred", grid_id_local, e)
        return None

def get_deviation_for_df(lat_local, lng_local, centroid_local):
    try:
        return int(geodesic((lat_local, lng_local), centroid_local).meters)
    except Exception as e:
        print("Exception occurred", e)
        return None

def get_drift_for_row(row_local):
    """get drift between pred coordinates and lookup coordinates"""
    pred_coor = (row_local['pred_lat'], row_local['pred_lng'])
    pred_drift = get_deviation_for_df(row_local["lookup_lat"], row_local["lookup_lng"], pred_coor)
    return pred_drift

def get_accuracy_for_df(drift_list_local):
    accuracy_dict_local = {100: 0, 200: 0, 500: 0, 1000: 0, 2000: 0, "rest": 0, "None": 0}
    for i in drift_list_local:
        if i is None or pd.isnull(i):
            accuracy_dict_local["None"] += 1
            continue

        if i < 100:
            accuracy_dict_local[100] += 1
        elif i < 200:
            accuracy_dict_local[200] += 1
        elif i < 500:
            accuracy_dict_local[500] += 1
        elif i < 1000:
            accuracy_dict_local[1000] += 1
        elif i < 2000:
            accuracy_dict_local[2000] += 1
        else:
            accuracy_dict_local["rest"] += 1

    accuracy_dict_local[200] += accuracy_dict_local[100]
    accuracy_dict_local[500] += accuracy_dict_local[200]
    accuracy_dict_local[1000] += accuracy_dict_local[500]
    accuracy_dict_local[2000] += accuracy_dict_local[1000]

    accuracy_dict_per_local = {100: 0, 200: 0, 500: 0, 1000: 0, 2000: 0, "rest": 0, "None": 0}
    for k, v in accuracy_dict_local.items():
        accuracy_dict_per_local[k] = float(v) / max(1, (accuracy_dict_local[2000] + accuracy_dict_local["rest"]))

    final_accuracy_dict_local = {'num': accuracy_dict_local, 'per': accuracy_dict_per_local}
    return final_accuracy_dict_local

test_df['pred_drift'] = test_df.apply(lambda row: get_drift_for_row(row), axis=1)
test_drift_acc_dict = get_accuracy_for_df(test_df['pred_drift'].tolist())

print(test_drift_acc_dict)

{'num': {100: 150212, 200: 169832, 500: 184939, 1000: 190925, 2000: 194258, 'rest': 10614, 'None': 0}, 'per': {100: 0.7331992658830879, 200: 0.8289663790073802, 500: 0.9027051036744894, 1000: 0.9319233472607287, 2000: 0.9481920418602835, 'rest': 0.0518079581397165, 'None': 0.0}}


In [77]:
def get_centroid_of_grid(grid_id_local): ### calssificarion_Roberta_RV_model@5 ref >=3
    try:
        return h3.h3_to_geo(grid_id_local)
    except Exception as e:
        print("Exception occurred", grid_id_local, e)
        return None

def get_deviation_for_df(lat_local, lng_local, centroid_local):
    try:
        return int(geodesic((lat_local, lng_local), centroid_local).meters)
    except Exception as e:
        print("Exception occurred", e)
        return None

def get_drift_for_row(row_local):
    """get drift between pred coordinates and lookup coordinates"""
    pred_coor = (row_local['pred_lat'], row_local['pred_lng'])
    pred_drift = get_deviation_for_df(row_local["lookup_lat"], row_local["lookup_lng"], pred_coor)
    return pred_drift

def get_accuracy_for_df(drift_list_local):
    accuracy_dict_local = {100: 0, 200: 0, 500: 0, 1000: 0, 2000: 0, "rest": 0, "None": 0}
    for i in drift_list_local:
        if i is None or pd.isnull(i):
            accuracy_dict_local["None"] += 1
            continue

        if i < 100:
            accuracy_dict_local[100] += 1
        elif i < 200:
            accuracy_dict_local[200] += 1
        elif i < 500:
            accuracy_dict_local[500] += 1
        elif i < 1000:
            accuracy_dict_local[1000] += 1
        elif i < 2000:
            accuracy_dict_local[2000] += 1
        else:
            accuracy_dict_local["rest"] += 1

    accuracy_dict_local[200] += accuracy_dict_local[100]
    accuracy_dict_local[500] += accuracy_dict_local[200]
    accuracy_dict_local[1000] += accuracy_dict_local[500]
    accuracy_dict_local[2000] += accuracy_dict_local[1000]

    accuracy_dict_per_local = {100: 0, 200: 0, 500: 0, 1000: 0, 2000: 0, "rest": 0, "None": 0}
    for k, v in accuracy_dict_local.items():
        accuracy_dict_per_local[k] = float(v) / max(1, (accuracy_dict_local[2000] + accuracy_dict_local["rest"]))

    final_accuracy_dict_local = {'num': accuracy_dict_local, 'per': accuracy_dict_per_local}
    return final_accuracy_dict_local

test_df['pred_drift'] = test_df.apply(lambda row: get_drift_for_row(row), axis=1)
test_drift_acc_dict = get_accuracy_for_df(test_df['pred_drift'].tolist())

print(test_drift_acc_dict)

{'num': {100: 148514, 200: 169054, 500: 184647, 1000: 190638, 2000: 194043, 'rest': 10829, 'None': 0}, 'per': {100: 0.7249111640438908, 200: 0.8251688859385372, 500: 0.9012798234995509, 1000: 0.9305224725682377, 2000: 0.9471426061150376, 'rest': 0.05285739388496232, 'None': 0.0}}


In [42]:
def get_centroid_of_grid(grid_id_local): ### calssificarion_Roberta_RV_model@1
    try:
        return h3.h3_to_geo(grid_id_local)
    except Exception as e:
        print("Exception occurred", grid_id_local, e)
        return None

def get_deviation_for_df(lat_local, lng_local, centroid_local):
    try:
        return int(geodesic((lat_local, lng_local), centroid_local).meters)
    except Exception as e:
        print("Exception occurred", e)
        return None

def get_drift_for_row(row_local):
    """get drift between pred coordinates and lookup coordinates"""
    pred_coor = (row_local['pred_lat'], row_local['pred_lng'])
    pred_drift = get_deviation_for_df(row_local["lookup_lat"], row_local["lookup_lng"], pred_coor)
    return pred_drift

def get_accuracy_for_df(drift_list_local):
    accuracy_dict_local = {100: 0, 200: 0, 500: 0, 1000: 0, 2000: 0, "rest": 0, "None": 0}
    for i in drift_list_local:
        if i is None or pd.isnull(i):
            accuracy_dict_local["None"] += 1
            continue

        if i < 100:
            accuracy_dict_local[100] += 1
        elif i < 200:
            accuracy_dict_local[200] += 1
        elif i < 500:
            accuracy_dict_local[500] += 1
        elif i < 1000:
            accuracy_dict_local[1000] += 1
        elif i < 2000:
            accuracy_dict_local[2000] += 1
        else:
            accuracy_dict_local["rest"] += 1

    accuracy_dict_local[200] += accuracy_dict_local[100]
    accuracy_dict_local[500] += accuracy_dict_local[200]
    accuracy_dict_local[1000] += accuracy_dict_local[500]
    accuracy_dict_local[2000] += accuracy_dict_local[1000]

    accuracy_dict_per_local = {100: 0, 200: 0, 500: 0, 1000: 0, 2000: 0, "rest": 0, "None": 0}
    for k, v in accuracy_dict_local.items():
        accuracy_dict_per_local[k] = float(v) / max(1, (accuracy_dict_local[2000] + accuracy_dict_local["rest"]))

    final_accuracy_dict_local = {'num': accuracy_dict_local, 'per': accuracy_dict_per_local}
    return final_accuracy_dict_local

test_df['pred_drift'] = test_df.apply(lambda row: get_drift_for_row(row), axis=1)
test_drift_acc_dict = get_accuracy_for_df(test_df['pred_drift'].tolist())

print(test_drift_acc_dict)

{'num': {100: 139797, 200: 160124, 500: 177172, 1000: 185094, 2000: 190234, 'rest': 14638, 'None': 0}, 'per': {100: 0.682362645944785, 200: 0.7815806942871647, 500: 0.8647936272404233, 1000: 0.9034616736303643, 2000: 0.9285505095864736, 'rest': 0.0714494904135265, 'None': 0.0}}


In [26]:
def get_centroid_of_grid(grid_id_local): ### calssificarion_Roberta_RV_model@10
    try:
        return h3.h3_to_geo(grid_id_local)
    except Exception as e:
        print("Exception occurred", grid_id_local, e)
        return None

def get_deviation_for_df(lat_local, lng_local, centroid_local):
    try:
        return int(geodesic((lat_local, lng_local), centroid_local).meters)
    except Exception as e:
        print("Exception occurred", e)
        return None

def get_drift_for_row(row_local):
    """get drift between pred coordinates and lookup coordinates"""
    pred_coor = (row_local['pred_lat'], row_local['pred_lng'])
    pred_drift = get_deviation_for_df(row_local["lookup_lat"], row_local["lookup_lng"], pred_coor)
    return pred_drift

def get_accuracy_for_df(drift_list_local):
    accuracy_dict_local = {100: 0, 200: 0, 500: 0, 1000: 0, 2000: 0, "rest": 0, "None": 0}
    for i in drift_list_local:
        if i is None or pd.isnull(i):
            accuracy_dict_local["None"] += 1
            continue

        if i < 100:
            accuracy_dict_local[100] += 1
        elif i < 200:
            accuracy_dict_local[200] += 1
        elif i < 500:
            accuracy_dict_local[500] += 1
        elif i < 1000:
            accuracy_dict_local[1000] += 1
        elif i < 2000:
            accuracy_dict_local[2000] += 1
        else:
            accuracy_dict_local["rest"] += 1

    accuracy_dict_local[200] += accuracy_dict_local[100]
    accuracy_dict_local[500] += accuracy_dict_local[200]
    accuracy_dict_local[1000] += accuracy_dict_local[500]
    accuracy_dict_local[2000] += accuracy_dict_local[1000]

    accuracy_dict_per_local = {100: 0, 200: 0, 500: 0, 1000: 0, 2000: 0, "rest": 0, "None": 0}
    for k, v in accuracy_dict_local.items():
        accuracy_dict_per_local[k] = float(v) / max(1, (accuracy_dict_local[2000] + accuracy_dict_local["rest"]))

    final_accuracy_dict_local = {'num': accuracy_dict_local, 'per': accuracy_dict_per_local}
    return final_accuracy_dict_local

test_df['pred_drift'] = test_df.apply(lambda row: get_drift_for_row(row), axis=1)
test_drift_acc_dict = get_accuracy_for_df(test_df['pred_drift'].tolist())

print(test_drift_acc_dict)

{'num': {100: 140474, 200: 165366, 500: 182844, 1000: 189317, 2000: 192848, 'rest': 12024, 'None': 0}, 'per': {100: 0.685667148268187, 200: 0.8071674020852044, 500: 0.8924792065289546, 1000: 0.9240745441055879, 2000: 0.9413096958100667, 'rest': 0.05869030418993323, 'None': 0.0}}


In [113]:
def get_centroid_of_grid(grid_id_local): ### calssificarion_Roberta_RV_model@3
    try:
        return h3.h3_to_geo(grid_id_local)
    except Exception as e:
        print("Exception occurred", grid_id_local, e)
        return None

def get_deviation_for_df(lat_local, lng_local, centroid_local):
    try:
        return int(geodesic((lat_local, lng_local), centroid_local).meters)
    except Exception as e:
        print("Exception occurred", e)
        return None

def get_drift_for_row(row_local):
    """get drift between pred coordinates and lookup coordinates"""
    pred_coor = (row_local['pred_lat'], row_local['pred_lng'])
    pred_drift = get_deviation_for_df(row_local["lookup_lat"], row_local["lookup_lng"], pred_coor)
    return pred_drift

def get_accuracy_for_df(drift_list_local):
    accuracy_dict_local = {100: 0, 200: 0, 500: 0, 1000: 0, 2000: 0, "rest": 0, "None": 0}
    for i in drift_list_local:
        if i is None or pd.isnull(i):
            accuracy_dict_local["None"] += 1
            continue

        if i < 100:
            accuracy_dict_local[100] += 1
        elif i < 200:
            accuracy_dict_local[200] += 1
        elif i < 500:
            accuracy_dict_local[500] += 1
        elif i < 1000:
            accuracy_dict_local[1000] += 1
        elif i < 2000:
            accuracy_dict_local[2000] += 1
        else:
            accuracy_dict_local["rest"] += 1

    accuracy_dict_local[200] += accuracy_dict_local[100]
    accuracy_dict_local[500] += accuracy_dict_local[200]
    accuracy_dict_local[1000] += accuracy_dict_local[500]
    accuracy_dict_local[2000] += accuracy_dict_local[1000]

    accuracy_dict_per_local = {100: 0, 200: 0, 500: 0, 1000: 0, 2000: 0, "rest": 0, "None": 0}
    for k, v in accuracy_dict_local.items():
        accuracy_dict_per_local[k] = float(v) / max(1, (accuracy_dict_local[2000] + accuracy_dict_local["rest"]))

    final_accuracy_dict_local = {'num': accuracy_dict_local, 'per': accuracy_dict_per_local}
    return final_accuracy_dict_local

test_df['pred_drift'] = test_df.apply(lambda row: get_drift_for_row(row), axis=1)
test_drift_acc_dict = get_accuracy_for_df(test_df['pred_drift'].tolist())

print(test_drift_acc_dict)

{'num': {100: 143359, 200: 165626, 500: 182438, 1000: 189137, 2000: 192875, 'rest': 11997, 'None': 0}, 'per': {100: 0.699749111640439, 200: 0.8084364871724784, 500: 0.8904974813542114, 1000: 0.9231959467374751, 2000: 0.9414414854152837, 'rest': 0.05855851458471631, 'None': 0.0}}


In [96]:
def get_centroid_of_grid(grid_id_local): ### calssificarion_Roberta_RV_model@5
    try:
        return h3.h3_to_geo(grid_id_local)
    except Exception as e:
        print("Exception occurred", grid_id_local, e)
        return None

def get_deviation_for_df(lat_local, lng_local, centroid_local):
    try:
        return int(geodesic((lat_local, lng_local), centroid_local).meters)
    except Exception as e:
        print("Exception occurred", e)
        return None

def get_drift_for_row(row_local):
    """get drift between pred coordinates and lookup coordinates"""
    pred_coor = (row_local['pred_lat'], row_local['pred_lng'])
    pred_drift = get_deviation_for_df(row_local["lookup_lat"], row_local["lookup_lng"], pred_coor)
    return pred_drift

def get_accuracy_for_df(drift_list_local):
    accuracy_dict_local = {100: 0, 200: 0, 500: 0, 1000: 0, 2000: 0, "rest": 0, "None": 0}
    for i in drift_list_local:
        if i is None or pd.isnull(i):
            accuracy_dict_local["None"] += 1
            continue

        if i < 100:
            accuracy_dict_local[100] += 1
        elif i < 200:
            accuracy_dict_local[200] += 1
        elif i < 500:
            accuracy_dict_local[500] += 1
        elif i < 1000:
            accuracy_dict_local[1000] += 1
        elif i < 2000:
            accuracy_dict_local[2000] += 1
        else:
            accuracy_dict_local["rest"] += 1

    accuracy_dict_local[200] += accuracy_dict_local[100]
    accuracy_dict_local[500] += accuracy_dict_local[200]
    accuracy_dict_local[1000] += accuracy_dict_local[500]
    accuracy_dict_local[2000] += accuracy_dict_local[1000]

    accuracy_dict_per_local = {100: 0, 200: 0, 500: 0, 1000: 0, 2000: 0, "rest": 0, "None": 0}
    for k, v in accuracy_dict_local.items():
        accuracy_dict_per_local[k] = float(v) / max(1, (accuracy_dict_local[2000] + accuracy_dict_local["rest"]))

    final_accuracy_dict_local = {'num': accuracy_dict_local, 'per': accuracy_dict_per_local}
    return final_accuracy_dict_local

test_df['pred_drift'] = test_df.apply(lambda row: get_drift_for_row(row), axis=1)
test_drift_acc_dict = get_accuracy_for_df(test_df['pred_drift'].tolist())

print(test_drift_acc_dict)

{'num': {100: 143144, 200: 166220, 500: 183206, 1000: 189569, 2000: 193153, 'rest': 11719, 'None': 0}, 'per': {100: 0.6986996758951931, 200: 0.8113358584872505, 500: 0.8942461634581592, 1000: 0.9253045804209458, 2000: 0.9427984302393689, 'rest': 0.05720156976063103, 'None': 0.0}}


In [62]:
def get_centroid_of_grid(grid_id_local): ### 2 epochs model_50 knn
    try:
        return h3.h3_to_geo(grid_id_local)
    except Exception as e:
        print("Exception occurred", grid_id_local, e)
        return None

def get_deviation_for_df(lat_local, lng_local, centroid_local):
    try:
        return int(geodesic((lat_local, lng_local), centroid_local).meters)
    except Exception as e:
        print("Exception occurred", e)
        return None

def get_drift_for_row(row_local):
    """get drift between pred coordinates and lookup coordinates"""
    pred_coor = (row_local['pred_lat'], row_local['pred_lng'])
    pred_drift = get_deviation_for_df(row_local["lookup_lat"], row_local["lookup_lng"], pred_coor)
    return pred_drift

def get_accuracy_for_df(drift_list_local):
    accuracy_dict_local = {100: 0, 200: 0, 500: 0, 1000: 0, 2000: 0, "rest": 0, "None": 0}
    for i in drift_list_local:
        if i is None or pd.isnull(i):
            accuracy_dict_local["None"] += 1
            continue

        if i < 100:
            accuracy_dict_local[100] += 1
        elif i < 200:
            accuracy_dict_local[200] += 1
        elif i < 500:
            accuracy_dict_local[500] += 1
        elif i < 1000:
            accuracy_dict_local[1000] += 1
        elif i < 2000:
            accuracy_dict_local[2000] += 1
        else:
            accuracy_dict_local["rest"] += 1

    accuracy_dict_local[200] += accuracy_dict_local[100]
    accuracy_dict_local[500] += accuracy_dict_local[200]
    accuracy_dict_local[1000] += accuracy_dict_local[500]
    accuracy_dict_local[2000] += accuracy_dict_local[1000]

    accuracy_dict_per_local = {100: 0, 200: 0, 500: 0, 1000: 0, 2000: 0, "rest": 0, "None": 0}
    for k, v in accuracy_dict_local.items():
        accuracy_dict_per_local[k] = float(v) / max(1, (accuracy_dict_local[2000] + accuracy_dict_local["rest"]))

    final_accuracy_dict_local = {'num': accuracy_dict_local, 'per': accuracy_dict_per_local}
    return final_accuracy_dict_local

test_df['pred_drift'] = test_df.apply(lambda row: get_drift_for_row(row), axis=1)
test_drift_acc_dict = get_accuracy_for_df(test_df['pred_drift'].tolist())

print(test_drift_acc_dict)

{'num': {100: 47085, 200: 80400, 500: 110079, 1000: 117055, 2000: 119517, 'rest': 85355, 'None': 0}, 'per': {100: 0.22982642820883284, 200: 0.39244015775703855, 500: 0.5373062204693663, 1000: 0.5713567495802258, 2000: 0.5833740091374127, 'rest': 0.4166259908625874, 'None': 0.0}}


In [26]:
def get_centroid_of_grid(grid_id_local): ### 4 epochs model
    try:
        return h3.h3_to_geo(grid_id_local)
    except Exception as e:
        print("Exception occurred", grid_id_local, e)
        return None

def get_deviation_for_df(lat_local, lng_local, centroid_local):
    try:
        return int(geodesic((lat_local, lng_local), centroid_local).meters)
    except Exception as e:
        print("Exception occurred", e)
        return None

def get_drift_for_row(row_local):
    """get drift between pred coordinates and lookup coordinates"""
    pred_coor = (row_local['pred_lat'], row_local['pred_lng'])
    pred_drift = get_deviation_for_df(row_local["lookup_lat"], row_local["lookup_lng"], pred_coor)
    return pred_drift

def get_accuracy_for_df(drift_list_local):
    accuracy_dict_local = {100: 0, 200: 0, 500: 0, 1000: 0, 2000: 0, "rest": 0, "None": 0}
    for i in drift_list_local:
        if i is None or pd.isnull(i):
            accuracy_dict_local["None"] += 1
            continue

        if i < 100:
            accuracy_dict_local[100] += 1
        elif i < 200:
            accuracy_dict_local[200] += 1
        elif i < 500:
            accuracy_dict_local[500] += 1
        elif i < 1000:
            accuracy_dict_local[1000] += 1
        elif i < 2000:
            accuracy_dict_local[2000] += 1
        else:
            accuracy_dict_local["rest"] += 1

    accuracy_dict_local[200] += accuracy_dict_local[100]
    accuracy_dict_local[500] += accuracy_dict_local[200]
    accuracy_dict_local[1000] += accuracy_dict_local[500]
    accuracy_dict_local[2000] += accuracy_dict_local[1000]

    accuracy_dict_per_local = {100: 0, 200: 0, 500: 0, 1000: 0, 2000: 0, "rest": 0, "None": 0}
    for k, v in accuracy_dict_local.items():
        accuracy_dict_per_local[k] = float(v) / max(1, (accuracy_dict_local[2000] + accuracy_dict_local["rest"]))

    final_accuracy_dict_local = {'num': accuracy_dict_local, 'per': accuracy_dict_per_local}
    return final_accuracy_dict_local

test_df['pred_drift'] = test_df.apply(lambda row: get_drift_for_row(row), axis=1)
test_drift_acc_dict = get_accuracy_for_df(test_df['pred_drift'].tolist())

print(test_drift_acc_dict)

{'num': {100: 60889, 200: 92330, 500: 123240, 1000: 130570, 2000: 132970, 'rest': 71902, 'None': 0}, 'per': {100: 0.2972050841501035, 200: 0.4506716388769573, 500: 0.6015463313678785, 1000: 0.6373247686360264, 2000: 0.6490394002108634, 'rest': 0.35096059978913663, 'None': 0.0}}


## 3epoch pan india - via triplets

In [28]:
def get_centroid_of_grid(grid_id_local):
    try:
        return h3.h3_to_geo(grid_id_local)
    except Exception as e:
        print("Exception occurred", grid_id_local, e)
        return None

def get_deviation_for_df(lat_local, lng_local, centroid_local):
    try:
        return int(geodesic((lat_local, lng_local), centroid_local).meters)
    except Exception as e:
        print("Exception occurred", e)
        return None

def get_drift_for_row(row_local):
    """get drift between pred coordinates and lookup coordinates"""
    pred_coor = (row_local['pred_lat'], row_local['pred_lng'])
    pred_drift = get_deviation_for_df(row_local["lookup_lat"], row_local["lookup_lng"], pred_coor)
    return pred_drift

def get_accuracy_for_df(drift_list_local):
    accuracy_dict_local = {100: 0, 200: 0, 500: 0, 1000: 0, 2000: 0, "rest": 0, "None": 0}
    for i in drift_list_local:
        if i is None or pd.isnull(i):
            accuracy_dict_local["None"] += 1
            continue

        if i < 100:
            accuracy_dict_local[100] += 1
        elif i < 200:
            accuracy_dict_local[200] += 1
        elif i < 500:
            accuracy_dict_local[500] += 1
        elif i < 1000:
            accuracy_dict_local[1000] += 1
        elif i < 2000:
            accuracy_dict_local[2000] += 1
        else:
            accuracy_dict_local["rest"] += 1

    accuracy_dict_local[200] += accuracy_dict_local[100]
    accuracy_dict_local[500] += accuracy_dict_local[200]
    accuracy_dict_local[1000] += accuracy_dict_local[500]
    accuracy_dict_local[2000] += accuracy_dict_local[1000]

    accuracy_dict_per_local = {100: 0, 200: 0, 500: 0, 1000: 0, 2000: 0, "rest": 0, "None": 0}
    for k, v in accuracy_dict_local.items():
        accuracy_dict_per_local[k] = float(v) / max(1, (accuracy_dict_local[2000] + accuracy_dict_local["rest"]))

    final_accuracy_dict_local = {'num': accuracy_dict_local, 'per': accuracy_dict_per_local}
    return final_accuracy_dict_local

test_df['pred_drift'] = test_df.apply(lambda row: get_drift_for_row(row), axis=1)
test_drift_acc_dict = get_accuracy_for_df(test_df['pred_drift'].tolist())

print(test_drift_acc_dict)

{'num': {100: 63276, 200: 95132, 500: 126197, 1000: 133711, 2000: 136120, 'rest': 68752, 'None': 0}, 'per': {100: 0.30885626147057677, 200: 0.4643484712405795, 500: 0.6159797336873756, 1000: 0.6526562927095942, 2000: 0.6644148541528369, 'rest': 0.3355851458471631, 'None': 0.0}}


## PAN_india_trained _via triplet for 2 epochs without city!!

In [72]:
def get_centroid_of_grid(grid_id_local):
    try:
        return h3.h3_to_geo(grid_id_local)
    except Exception as e:
        print("Exception occurred", grid_id_local, e)
        return None

def get_deviation_for_df(lat_local, lng_local, centroid_local):
    try:
        return int(geodesic((lat_local, lng_local), centroid_local).meters)
    except Exception as e:
        print("Exception occurred", e)
        return None

def get_drift_for_row(row_local):
    """get drift between pred coordinates and lookup coordinates"""
    pred_coor = (row_local['pred_lat'], row_local['pred_lng'])
    pred_drift = get_deviation_for_df(row_local["lookup_lat"], row_local["lookup_lng"], pred_coor)
    return pred_drift

def get_accuracy_for_df(drift_list_local):
    accuracy_dict_local = {100: 0, 200: 0, 500: 0, 1000: 0, 2000: 0, "rest": 0, "None": 0}
    for i in drift_list_local:
        if i is None or pd.isnull(i):
            accuracy_dict_local["None"] += 1
            continue

        if i < 100:
            accuracy_dict_local[100] += 1
        elif i < 200:
            accuracy_dict_local[200] += 1
        elif i < 500:
            accuracy_dict_local[500] += 1
        elif i < 1000:
            accuracy_dict_local[1000] += 1
        elif i < 2000:
            accuracy_dict_local[2000] += 1
        else:
            accuracy_dict_local["rest"] += 1

    accuracy_dict_local[200] += accuracy_dict_local[100]
    accuracy_dict_local[500] += accuracy_dict_local[200]
    accuracy_dict_local[1000] += accuracy_dict_local[500]
    accuracy_dict_local[2000] += accuracy_dict_local[1000]

    accuracy_dict_per_local = {100: 0, 200: 0, 500: 0, 1000: 0, 2000: 0, "rest": 0, "None": 0}
    for k, v in accuracy_dict_local.items():
        accuracy_dict_per_local[k] = float(v) / max(1, (accuracy_dict_local[2000] + accuracy_dict_local["rest"]))

    final_accuracy_dict_local = {'num': accuracy_dict_local, 'per': accuracy_dict_per_local}
    return final_accuracy_dict_local

test_df['pred_drift'] = test_df.apply(lambda row: get_drift_for_row(row), axis=1)
test_drift_acc_dict = get_accuracy_for_df(test_df['pred_drift'].tolist())

print(test_drift_acc_dict)

{'num': {100: 67646, 200: 100637, 500: 131630, 1000: 139032, 2000: 141574, 'rest': 63298, 'None': 0}, 'per': {100: 0.3301866531297591, 200: 0.4912189074153618, 500: 0.6424987309149127, 1000: 0.6786286071303057, 2000: 0.6910363544066539, 'rest': 0.3089636455933461, 'None': 0.0}}


In [51]:
### triplet 2 epochs!! 2.2

In [65]:
def get_centroid_of_grid(grid_id_local): ### this one 4:12
    try:
        return h3.h3_to_geo(grid_id_local)
    except Exception as e:
        print("Exception occurred", grid_id_local, e)
        return None

def get_deviation_for_df(lat_local, lng_local, centroid_local):
    try:
        return int(geodesic((lat_local, lng_local), centroid_local).meters)
    except Exception as e:
        print("Exception occurred", e)
        return None

def get_drift_for_row(row_local):
    """get drift between pred coordinates and lookup coordinates"""
    pred_coor = (row_local['pred_lat'], row_local['pred_lng'])
    pred_drift = get_deviation_for_df(row_local["lookup_lat"], row_local["lookup_lng"], pred_coor)
    return pred_drift

def get_accuracy_for_df(drift_list_local):
    accuracy_dict_local = {100: 0, 200: 0, 500: 0, 1000: 0, 2000: 0, "rest": 0, "None": 0}
    for i in drift_list_local:
        if i is None or pd.isnull(i):
            accuracy_dict_local["None"] += 1
            continue

        if i < 100:
            accuracy_dict_local[100] += 1
        elif i < 200:
            accuracy_dict_local[200] += 1
        elif i < 500:
            accuracy_dict_local[500] += 1
        elif i < 1000:
            accuracy_dict_local[1000] += 1
        elif i < 2000:
            accuracy_dict_local[2000] += 1
        else:
            accuracy_dict_local["rest"] += 1

    accuracy_dict_local[200] += accuracy_dict_local[100]
    accuracy_dict_local[500] += accuracy_dict_local[200]
    accuracy_dict_local[1000] += accuracy_dict_local[500]
    accuracy_dict_local[2000] += accuracy_dict_local[1000]

    accuracy_dict_per_local = {100: 0, 200: 0, 500: 0, 1000: 0, 2000: 0, "rest": 0, "None": 0}
    for k, v in accuracy_dict_local.items():
        accuracy_dict_per_local[k] = float(v) / max(1, (accuracy_dict_local[2000] + accuracy_dict_local["rest"]))

    final_accuracy_dict_local = {'num': accuracy_dict_local, 'per': accuracy_dict_per_local}
    return final_accuracy_dict_local

test_df['pred_drift'] = test_df.apply(lambda row: get_drift_for_row(row), axis=1)
test_drift_acc_dict = get_accuracy_for_df(test_df['pred_drift'].tolist())

print(test_drift_acc_dict)

{'num': {100: 67646, 200: 100637, 500: 131630, 1000: 139032, 2000: 141574, 'rest': 63298, 'None': 0}, 'per': {100: 0.3301866531297591, 200: 0.4912189074153618, 500: 0.6424987309149127, 1000: 0.6786286071303057, 2000: 0.6910363544066539, 'rest': 0.3089636455933461, 'None': 0.0}}


In [50]:
def get_centroid_of_grid(grid_id_local):
    try:
        return h3.h3_to_geo(grid_id_local)
    except Exception as e:
        print("Exception occurred", grid_id_local, e)
        return None

def get_deviation_for_df(lat_local, lng_local, centroid_local):
    try:
        return int(geodesic((lat_local, lng_local), centroid_local).meters)
    except Exception as e:
        print("Exception occurred", e)
        return None

def get_drift_for_row(row_local):
    """get drift between pred coordinates and lookup coordinates"""
    pred_coor = (row_local['pred_lat'], row_local['pred_lng'])
    pred_drift = get_deviation_for_df(row_local["lookup_lat"], row_local["lookup_lng"], pred_coor)
    return pred_drift

def get_accuracy_for_df(drift_list_local):
    accuracy_dict_local = {100: 0, 200: 0, 500: 0, 1000: 0, 2000: 0, "rest": 0, "None": 0}
    for i in drift_list_local:
        if i is None or pd.isnull(i):
            accuracy_dict_local["None"] += 1
            continue

        if i < 100:
            accuracy_dict_local[100] += 1
        elif i < 200:
            accuracy_dict_local[200] += 1
        elif i < 500:
            accuracy_dict_local[500] += 1
        elif i < 1000:
            accuracy_dict_local[1000] += 1
        elif i < 2000:
            accuracy_dict_local[2000] += 1
        else:
            accuracy_dict_local["rest"] += 1

    accuracy_dict_local[200] += accuracy_dict_local[100]
    accuracy_dict_local[500] += accuracy_dict_local[200]
    accuracy_dict_local[1000] += accuracy_dict_local[500]
    accuracy_dict_local[2000] += accuracy_dict_local[1000]

    accuracy_dict_per_local = {100: 0, 200: 0, 500: 0, 1000: 0, 2000: 0, "rest": 0, "None": 0}
    for k, v in accuracy_dict_local.items():
        accuracy_dict_per_local[k] = float(v) / max(1, (accuracy_dict_local[2000] + accuracy_dict_local["rest"]))

    final_accuracy_dict_local = {'num': accuracy_dict_local, 'per': accuracy_dict_per_local}
    return final_accuracy_dict_local

test_df['pred_drift'] = test_df.apply(lambda row: get_drift_for_row(row), axis=1)
test_drift_acc_dict = get_accuracy_for_df(test_df['pred_drift'].tolist())

print(test_drift_acc_dict)

{'num': {100: 59548, 200: 91017, 500: 122316, 1000: 129988, 2000: 132396, 'rest': 72476, 'None': 0}, 'per': {100: 0.29065953375766335, 200: 0.4442627591862236, 500: 0.5970361982115663, 1000: 0.6344839704791284, 2000: 0.6462376508258815, 'rest': 0.3537623491741185, 'None': 0.0}}


## PAN iNdia model , without city  

In [81]:
def get_centroid_of_grid(grid_id_local): ###BASE 4:52
    try:
        return h3.h3_to_geo(grid_id_local)
    except Exception as e:
        print("Exception occurred", grid_id_local, e)
        return None

def get_deviation_for_df(lat_local, lng_local, centroid_local):
    try:
        return int(geodesic((lat_local, lng_local), centroid_local).meters)
    except Exception as e:
        print("Exception occurred", e)
        return None

def get_drift_for_row(row_local):
    """get drift between pred coordinates and lookup coordinates"""
    pred_coor = (row_local['pred_lat'], row_local['pred_lng'])
    pred_drift = get_deviation_for_df(row_local["lookup_lat"], row_local["lookup_lng"], pred_coor)
    return pred_drift

def get_accuracy_for_df(drift_list_local):
    accuracy_dict_local = {100: 0, 200: 0, 500: 0, 1000: 0, 2000: 0, "rest": 0, "None": 0}
    for i in drift_list_local:
        if i is None or pd.isnull(i):
            accuracy_dict_local["None"] += 1
            continue

        if i < 100:
            accuracy_dict_local[100] += 1
        elif i < 200:
            accuracy_dict_local[200] += 1
        elif i < 500:
            accuracy_dict_local[500] += 1
        elif i < 1000:
            accuracy_dict_local[1000] += 1
        elif i < 2000:
            accuracy_dict_local[2000] += 1
        else:
            accuracy_dict_local["rest"] += 1

    accuracy_dict_local[200] += accuracy_dict_local[100]
    accuracy_dict_local[500] += accuracy_dict_local[200]
    accuracy_dict_local[1000] += accuracy_dict_local[500]
    accuracy_dict_local[2000] += accuracy_dict_local[1000]

    accuracy_dict_per_local = {100: 0, 200: 0, 500: 0, 1000: 0, 2000: 0, "rest": 0, "None": 0}
    for k, v in accuracy_dict_local.items():
        accuracy_dict_per_local[k] = float(v) / max(1, (accuracy_dict_local[2000] + accuracy_dict_local["rest"]))

    final_accuracy_dict_local = {'num': accuracy_dict_local, 'per': accuracy_dict_per_local}
    return final_accuracy_dict_local

test_df['pred_drift'] = test_df.apply(lambda row: get_drift_for_row(row), axis=1)
test_drift_acc_dict = get_accuracy_for_df(test_df['pred_drift'].tolist())

print(test_drift_acc_dict)

{'num': {100: 49488, 200: 72978, 500: 104852, 1000: 123875, 2000: 138099, 'rest': 66773, 'None': 0}, 'per': {100: 0.24155570307313834, 200: 0.3562126596118552, 500: 0.5117927291186692, 1000: 0.6046458276387208, 2000: 0.6740745441055879, 'rest': 0.32592545589441213, 'None': 0.0}}


### only pan india

In [40]:
def get_centroid_of_grid(grid_id_local):
    try:
        return h3.h3_to_geo(grid_id_local)
    except Exception as e:
        print("Exception occurred", grid_id_local, e)
        return None

def get_deviation_for_df(lat_local, lng_local, centroid_local):
    try:
        return int(geodesic((lat_local, lng_local), centroid_local).meters)
    except Exception as e:
        print("Exception occurred", e)
        return None

def get_drift_for_row(row_local):
    """get drift between pred coordinates and lookup coordinates"""
    pred_coor = (row_local['pred_lat'], row_local['pred_lng'])
    pred_drift = get_deviation_for_df(row_local["lookup_lat"], row_local["lookup_lng"], pred_coor)
    return pred_drift

def get_accuracy_for_df(drift_list_local):
    accuracy_dict_local = {100: 0, 200: 0, 500: 0, 1000: 0, 2000: 0, "rest": 0, "None": 0}
    for i in drift_list_local:
        if i is None or pd.isnull(i):
            accuracy_dict_local["None"] += 1
            continue

        if i < 100:
            accuracy_dict_local[100] += 1
        elif i < 200:
            accuracy_dict_local[200] += 1
        elif i < 500:
            accuracy_dict_local[500] += 1
        elif i < 1000:
            accuracy_dict_local[1000] += 1
        elif i < 2000:
            accuracy_dict_local[2000] += 1
        else:
            accuracy_dict_local["rest"] += 1

    accuracy_dict_local[200] += accuracy_dict_local[100]
    accuracy_dict_local[500] += accuracy_dict_local[200]
    accuracy_dict_local[1000] += accuracy_dict_local[500]
    accuracy_dict_local[2000] += accuracy_dict_local[1000]

    accuracy_dict_per_local = {100: 0, 200: 0, 500: 0, 1000: 0, 2000: 0, "rest": 0, "None": 0}
    for k, v in accuracy_dict_local.items():
        accuracy_dict_per_local[k] = float(v) / max(1, (accuracy_dict_local[2000] + accuracy_dict_local["rest"]))

    final_accuracy_dict_local = {'num': accuracy_dict_local, 'per': accuracy_dict_per_local}
    return final_accuracy_dict_local

test_df['pred_drift'] = test_df.apply(lambda row: get_drift_for_row(row), axis=1)
test_drift_acc_dict = get_accuracy_for_df(test_df['pred_drift'].tolist())

print(test_drift_acc_dict)

{'num': {100: 49488, 200: 72978, 500: 104852, 1000: 123875, 2000: 138099, 'rest': 66773, 'None': 0}, 'per': {100: 0.24155570307313834, 200: 0.3562126596118552, 500: 0.5117927291186692, 1000: 0.6046458276387208, 2000: 0.6740745441055879, 'rest': 0.32592545589441213, 'None': 0.0}}


## On 12800 samples 

In [67]:
def get_centroid_of_grid(grid_id_local):
    try:
        return h3.h3_to_geo(grid_id_local)
    except Exception as e:
        print("Exception occurred", grid_id_local, e)
        return None

def get_deviation_for_df(lat_local, lng_local, centroid_local):
    try:
        return int(geodesic((lat_local, lng_local), centroid_local).meters)
    except Exception as e:
        print("Exception occurred", e)
        return None

def get_drift_for_row(row_local):
    """get drift between pred coordinates and lookup coordinates"""
    pred_coor = (row_local['pred_lat'], row_local['pred_lng'])
    pred_drift = get_deviation_for_df(row_local["lookup_lat"], row_local["lookup_lng"], pred_coor)
    return pred_drift

def get_accuracy_for_df(drift_list_local):
    accuracy_dict_local = {100: 0, 200: 0, 500: 0, 1000: 0, 2000: 0, "rest": 0, "None": 0}
    for i in drift_list_local:
        if i is None or pd.isnull(i):
            accuracy_dict_local["None"] += 1
            continue

        if i < 100:
            accuracy_dict_local[100] += 1
        elif i < 200:
            accuracy_dict_local[200] += 1
        elif i < 500:
            accuracy_dict_local[500] += 1
        elif i < 1000:
            accuracy_dict_local[1000] += 1
        elif i < 2000:
            accuracy_dict_local[2000] += 1
        else:
            accuracy_dict_local["rest"] += 1

    accuracy_dict_local[200] += accuracy_dict_local[100]
    accuracy_dict_local[500] += accuracy_dict_local[200]
    accuracy_dict_local[1000] += accuracy_dict_local[500]
    accuracy_dict_local[2000] += accuracy_dict_local[1000]

    accuracy_dict_per_local = {100: 0, 200: 0, 500: 0, 1000: 0, 2000: 0, "rest": 0, "None": 0}
    for k, v in accuracy_dict_local.items():
        accuracy_dict_per_local[k] = float(v) / max(1, (accuracy_dict_local[2000] + accuracy_dict_local["rest"]))

    final_accuracy_dict_local = {'num': accuracy_dict_local, 'per': accuracy_dict_per_local}
    return final_accuracy_dict_local

test_df2['pred_drift'] = test_df2.apply(lambda row: get_drift_for_row(row), axis=1)
test_drift_acc_dict = get_accuracy_for_df(test_df2['pred_drift'].tolist())

print(test_drift_acc_dict)

{'num': {100: 4147, 200: 6650, 500: 8811, 1000: 9088, 2000: 9156, 'rest': 3644, 'None': 0}, 'per': {100: 0.323984375, 200: 0.51953125, 500: 0.688359375, 1000: 0.71, 2000: 0.7153125, 'rest': 0.2846875, 'None': 0.0}}


In [77]:
# def get_centroid_of_grid(grid_id_local):
#     try:
#         return h3.h3_to_geo(grid_id_local)
#     except Exception as e:
#         print("Exception occurred", grid_id_local, e)
#         return None

def get_deviation_for_df(lat_local, lng_local, centroid_local):
    try:
        return int(geodesic((lat_local, lng_local), centroid_local).meters)
    except Exception as e:
        print("Exception occurred", e)
        return None

def get_drift_for_row(row_local):
    """get drift between pred coordinates and lookup coordinates"""
    pred_coor = (row_local['pred_lat'], row_local['pred_lng'])
    pred_drift = get_deviation_for_df(row_local["lookup_lat"], row_local["lookup_lng"], pred_coor)
    return pred_drift

def get_accuracy_for_df(drift_list_local):
    accuracy_dict_local = {100: 0, 200: 0, 500: 0, 1000: 0, 2000: 0, "rest": 0, "None": 0}
    for i in drift_list_local:
        if i is None or pd.isnull(i):
            accuracy_dict_local["None"] += 1
            continue

        if i < 100:
            accuracy_dict_local[100] += 1
        elif i < 200:
            accuracy_dict_local[200] += 1
        elif i < 500:
            accuracy_dict_local[500] += 1
        elif i < 1000:
            accuracy_dict_local[1000] += 1
        elif i < 2000:
            accuracy_dict_local[2000] += 1
        else:
            accuracy_dict_local["rest"] += 1

    accuracy_dict_local[200] += accuracy_dict_local[100]
    accuracy_dict_local[500] += accuracy_dict_local[200]
    accuracy_dict_local[1000] += accuracy_dict_local[500]
    accuracy_dict_local[2000] += accuracy_dict_local[1000]

    accuracy_dict_per_local = {100: 0, 200: 0, 500: 0, 1000: 0, 2000: 0, "rest": 0, "None": 0}
    for k, v in accuracy_dict_local.items():
        accuracy_dict_per_local[k] = float(v) / max(1, (accuracy_dict_local[2000] + accuracy_dict_local["rest"]))

    final_accuracy_dict_local = {'num': accuracy_dict_local, 'per': accuracy_dict_per_local}
    return final_accuracy_dict_local

test_df2['pred_drift'] = test_df2.apply(lambda row: get_drift_for_row(row), axis=1)
test_drift_acc_dict = get_accuracy_for_df(test_df2['pred_drift'].tolist())

print(test_drift_acc_dict)

{'num': {100: 4011, 200: 6606, 500: 8620, 1000: 8864, 2000: 8922, 'rest': 3878, 'None': 0}, 'per': {100: 0.313359375, 200: 0.51609375, 500: 0.6734375, 1000: 0.6925, 2000: 0.69703125, 'rest': 0.30296875, 'None': 0.0}}


In [104]:
# def get_centroid_of_grid(grid_id_local):
#     try:
#         return h3.h3_to_geo(grid_id_local)
#     except Exception as e:
#         print("Exception occurred", grid_id_local, e)
#         return None

def get_deviation_for_df(lat_local, lng_local, centroid_local):
    try:
        return int(geodesic((lat_local, lng_local), centroid_local).meters)
    except Exception as e:
        print("Exception occurred", e)
        return None

def get_drift_for_row(row_local):
    """get drift between pred coordinates and lookup coordinates"""
    pred_coor = (row_local['pred_lat'], row_local['pred_lng'])
    pred_drift = get_deviation_for_df(row_local["lookup_lat"], row_local["lookup_lng"], pred_coor)
    return pred_drift

def get_accuracy_for_df(drift_list_local):
    accuracy_dict_local = {100: 0, 200: 0, 500: 0, 1000: 0, 2000: 0, "rest": 0, "None": 0}
    for i in drift_list_local:
        if i is None or pd.isnull(i):
            accuracy_dict_local["None"] += 1
            continue

        if i < 100:
            accuracy_dict_local[100] += 1
        elif i < 200:
            accuracy_dict_local[200] += 1
        elif i < 500:
            accuracy_dict_local[500] += 1
        elif i < 1000:
            accuracy_dict_local[1000] += 1
        elif i < 2000:
            accuracy_dict_local[2000] += 1
        else:
            accuracy_dict_local["rest"] += 1

    accuracy_dict_local[200] += accuracy_dict_local[100]
    accuracy_dict_local[500] += accuracy_dict_local[200]
    accuracy_dict_local[1000] += accuracy_dict_local[500]
    accuracy_dict_local[2000] += accuracy_dict_local[1000]

    accuracy_dict_per_local = {100: 0, 200: 0, 500: 0, 1000: 0, 2000: 0, "rest": 0, "None": 0}
    for k, v in accuracy_dict_local.items():
        accuracy_dict_per_local[k] = float(v) / max(1, (accuracy_dict_local[2000] + accuracy_dict_local["rest"]))

    final_accuracy_dict_local = {'num': accuracy_dict_local, 'per': accuracy_dict_per_local}
    return final_accuracy_dict_local

test_df['pred_drift'] = test_df.apply(lambda row: get_drift_for_row(row), axis=1)
test_drift_acc_dict = get_accuracy_for_df(test_df['pred_drift'].tolist())

print(test_drift_acc_dict)

{'num': {100: 54036, 200: 81509, 500: 108417, 1000: 114313, 2000: 116312, 'rest': 88560, 'None': 0}, 'per': {100: 0.2637549299074544, 200: 0.3978532937639111, 500: 0.5291938381037916, 1000: 0.5579727830059744, 2000: 0.5677300948885158, 'rest': 0.4322699051114842, 'None': 0.0}}
